In [3]:
# import libraries
import simpy
import random
from datetime import timedelta

In [4]:
"""
Orders arrival at supplier.

Covers:

- Waiting for other processes
- Resources: Resource

Scenario:
  A supplier has a limited number of processing machines and defines
  a processing processes that takes some (random) time.

  Order processes arrive at the supplier at a random time. If one processing
  machine is available, they start the preparing process and wait for it
  to finish. If not, they wait until they can use one.

"""
LAMBDA 		= 20
RANDOM_SEED 	= 42
NUM_MACHINES 	= 2		# Number of machines at the supplier
PROCESSTIME 	= 4      	# Minutes it takes to prepare an order
T_INTER 		= 60/ LAMBDA 	# Create an order every ~3 minutes assume rate 20
SIM_TIME 	= 480     	# Simulation time in minutes
init_orders = 5  # Create n initial orders

In [11]:
class OrderPreparing(object):
    """A supplier has a limited number of machines (``NUM_MACHINES``) to
    Prepare orders in parallel.

    orders have to request one of the machines. When they got one, they
    can start the preparing processes and wait for it to finish (which
    takes ``processtime`` minutes).

    """
    def __init__(self, env, num_machines, processtime):
        self.env = env
        self.machine = simpy.Resource(env, num_machines)
        self. processtime = processtime

    def prepare(self, order):
        """The preparing processes. It takes an ``order`` processes and tries
        to prepare it."""
        yield self.env.timeout(PROCESSTIME)
        print("Supplier prepared %s." %(order))


def order(env, name, sp):
    """The order process (each order has a ``name``) arrives at the supplier
    (``sp``) and requests a processing machine.

    It then starts the preparing process, waits for it to finish and
    leaves to never come back ...

    """
    
    print('Arrival    503: %s at %s.' % (name, str(timedelta(minutes=env.now))[:-3]))
    with sp.machine.request() as request:
        yield request

        print('Processing 603: %s at %s.' % (name, str(timedelta(minutes=env.now))[:-3]))
        yield env.process(sp.prepare(name))

        print('Leaving    703: %s at %s.' % (name, str(timedelta(minutes=env.now))[:-3]))


def setup(env, num_machines, processtime, t_inter, init_orders):
    """Create a supplier, a number of initial orders and keep creating orders
    approx. every ``t_inter`` minutes."""
    # Create the supplier
    supplier = OrderPreparing(env, num_machines, processtime)

    # Create n initial orders (number+1 to start from 1)
    for i in (number+1 for number in range(init_orders)):
            env.process(order(env, 'order_id= %d' % i, supplier))

    # Create more orders while the simulation is running
    while True:
        yield env.timeout(random.randint(t_inter - 2, t_inter + 2))
        i += 1
        env.process(order(env, 'order_id= %d' % i, supplier))


# Setup and start the simulation
print('Starting process:')
random.seed(RANDOM_SEED)  # This helps reproducing the results

# Create an environment and start the setup process
env = simpy.Environment()
env.process(setup(env, NUM_MACHINES, PROCESSTIME, T_INTER, init_orders))

# Execute!
env.run(until=SIM_TIME)

Starting process:
Arrival    503: order_id= 1 at 0:00.
Arrival    503: order_id= 2 at 0:00.
Arrival    503: order_id= 3 at 0:00.
Arrival    503: order_id= 4 at 0:00.
Arrival    503: order_id= 5 at 0:00.
Processing 603: order_id= 1 at 0:00.
Processing 603: order_id= 2 at 0:00.
Arrival    503: order_id= 6 at 0:01.
Arrival    503: order_id= 7 at 0:02.
Supplier prepared order_id= 1.
Supplier prepared order_id= 2.
Leaving    703: order_id= 1 at 0:04.
Leaving    703: order_id= 2 at 0:04.
Processing 603: order_id= 3 at 0:04.
Processing 603: order_id= 4 at 0:04.
Arrival    503: order_id= 8 at 0:05.
Arrival    503: order_id= 9 at 0:07.
Supplier prepared order_id= 3.
Supplier prepared order_id= 4.
Leaving    703: order_id= 3 at 0:08.
Leaving    703: order_id= 4 at 0:08.
Processing 603: order_id= 5 at 0:08.
Processing 603: order_id= 6 at 0:08.
Arrival    503: order_id= 10 at 0:09.
Arrival    503: order_id= 11 at 0:11.
Supplier prepared order_id= 5.
Supplier prepared order_id= 6.
Arrival    503: o